<a href="https://colab.research.google.com/github/AdamMcCloskey381/RTA_project/blob/main/MLPR_flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [ ]:
### załadowanie danych
df = pd.DataFrame()
for i in range(118, 1555):
  try:
    df1 = pd.read_csv("biegi_{}.csv".format(i))
    df = df.append(df1)
  except:
    pass

In [ ]:
### zastąpienie części braków danych
df["Klub"].isnull().sum()
df["Klub"] = df["Klub"].replace(np.nan, "Brak klubu")

In [ ]:
### usunięcie pozostałych braków danych
df = df.dropna()

In [ ]:
df.isnull().sum()

Miejsce        0
Nr             0
Nazwisko       0
Imię           0
Miasto         0
Klub           0
Rocznik        0
Kategoria      0
Czas netto     0
Czas brutto    0
Data           0
Rok            0
Miesiąc        0
Płeć           0
dtype: int64

In [ ]:
from datetime import date
df["New_Klub"] = 0
df.loc[df["Klub"] != "Brak klubu", "New_Klub"] = 1
df["Wiek"] = date.today().year - df["Rocznik"]

In [ ]:
df_final = df.loc[:, ["Czas netto", "Miesiąc", "New_Klub", "Wiek", "Płeć"]]
df_final["Miesiąc"], df_final["Płeć"] = df_final["Miesiąc"].astype("float64"), df_final["Płeć"].astype("category") 
df_final["New_Klub"], df_final["Wiek"] = df_final["New_Klub"].astype("float64"), df_final["Wiek"].astype("float64")

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_final["Płeć"] = le.fit_transform(df_final["Płeć"])

In [ ]:
MLPR = MLPRegressor(hidden_layer_sizes = (4, ), learning_rate = "adaptive", learning_rate_init= 0.001, max_iter = 1000)

In [ ]:
x = df_final.loc[:, df_final.columns != "Czas netto"]
y = df_final.loc[:, df_final.columns == "Czas netto"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 11)

In [ ]:
y_train = np.ravel(y_train)
MLPR.fit(x_train, y_train)
pred = MLPR.predict(x_test)

In [ ]:
r2_score(y_test, pred)

0.28257027056069295

In [ ]:
import pickle

with open("MLPR.pkl", "wb") as fh:
  pickle.dump(MLPR, fh)

In [ ]:
df_final.dtypes

Czas netto    float64
Miesiąc       float64
New_Klub      float64
Wiek          float64
Płeć            int64
dtype: object

In [103]:
%%file MLPR.py

from flask import Flask, request
import pickle
import numpy as np
from sklearn.neural_network import MLPRegressor

app = Flask(__name__)


@app.route("/MLPR", methods=['GET'])
def make_prediction():
    Miesiac = request.args.get("Miesiąc", 1, type=float)
    New_Klub = request.args.get("New_Klub", 0, type=float)
    Wiek = request.args.get("Wiek", 30, type=float)
    Plec = request.args.get("Płeć", 1, type=int)
    data = np.array([Miesiac, New_Klub, Wiek, Plec]).reshape(1, -1)
    
    with open("MLPR.pkl", "rb") as fh:    
        model = pickle.load(fh)
    
    pred = model.predict(data)[0]
    
    return {"prediction": pred, "features": {"Miesiac": Miesiac, "New_Klub": New_Klub, "Wiek": Wiek, "Plec": Plec}}

if __name__ == '__main__':
    app.run()

Overwriting MLPR.py


In [104]:
import subprocess
p = subprocess.Popen(["python", "MLPR.py"])

In [106]:
import requests
response = requests.get("http://127.0.0.1:5000/MLPR", params={"Miesiąc": 1, "New_Klub": 1, "Wiek": 30, "Płeć": 1})
response.json()

{'features': {'Miesiac': 1.0, 'New_Klub': 1.0, 'Plec': 1, 'Wiek': 30.0},
 'prediction': 21.59408703911815}